In [1]:
!pip install transformers datasets torch scikit-learn matplotlib seaborn pandas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_scheduler
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix


# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the dataset from Google Drive
file_path = '/content/drive/MyDrive/Dataset/website_dataset.csv'  # Replace with your dataset's path in Drive
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
# print("Dataset Preview:")
# print(data.head())

# Check the number of categories and the distribution of samples in each category
category_counts = data['Category'].value_counts()

# Display the number of categories and their sample counts
num_categories = len(category_counts)
# print(f"\nNumber of categories: {num_categories}")
# print("\nSample count per category:")
# print(category_counts)

# Plot the distribution of categories
# plt.figure(figsize=(12, 6))
# sns.barplot(x=category_counts.index, y=category_counts.values, palette="viridis")
# plt.title("Distribution of Samples Across Categories")
# plt.xlabel("Category")
# plt.ylabel("Number of Samples")
# plt.xticks(rotation=45, ha="right")
# plt.tight_layout()
# plt.show()



import re

# Combine `url` and `description` into a single text feature
data['text'] = data['URL'] + " " + data['Description']

# Clean the text (optional: depending on the dataset)
def clean_text(text):
    # Remove special characters and multiple spaces
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

data['text'] = data['text'].apply(clean_text)

# Display a few samples of preprocessed text
# print("Preprocessed Text Samples:")
# print(data[['text', 'Category']].head())


import os
import json
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the `category` column
data['label'] = label_encoder.fit_transform(data['Category'])

# Create a mapping of categories to numerical labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Create a reverse mapping (numerical labels to original category names)
reverse_label_map = {value: key for key, value in label_mapping.items()}

# Display mappings
#print("\nCategory to Label Mapping:")
# for category, label in label_mapping.items():
#     print(f"{category} -> {label}")

# print("\nLabel to Category Mapping (Reverse):")
# for label, category in reverse_label_map.items():
#     print(f"{label} -> {category}")



from sklearn.model_selection import train_test_split

# Step 1: Split 75% for training and 25% for test+validation
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    data['text'], data['label'], test_size=0.25, stratify=data['label'], random_state=42
)

# Step 2: Split the remaining 25% into 12.5% test and 12.5% validation
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42
)

# Display the number of samples in each set
# print(f"Training Set Size: {len(train_texts)}")
# print(f"Validation Set Size: {len(val_texts)}")
# print(f"Test Set Size: {len(test_texts)}")

# Check class distributions to verify stratification
train_class_distribution = pd.Series(train_labels).value_counts()
val_class_distribution = pd.Series(val_labels).value_counts()
test_class_distribution = pd.Series(test_labels).value_counts()

# print("\nClass Distribution in Training Set:")
# print(train_class_distribution)

# print("\nClass Distribution in Validation Set:")
# print(val_class_distribution)

# print("\nClass Distribution in Test Set:")
# print(test_class_distribution)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Mounted at /content/drive


**Code to Train Using RoBERTa**

In [2]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load the pre-trained RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
print("RoBERTa Tokenizer Loaded.")

# Tokenize the training, validation, and test texts
train_encodings = tokenizer(
    list(train_texts), truncation=True, padding=True, max_length=128
)
val_encodings = tokenizer(
    list(val_texts), truncation=True, padding=True, max_length=128
)
test_encodings = tokenizer(
    list(test_texts), truncation=True, padding=True, max_length=128
)

print("Tokenization Completed for Training, Validation, and Test Sets.")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

RoBERTa Tokenizer Loaded.
Tokenization Completed for Training, Validation, and Test Sets.


In [3]:
# Define the dataset class (reuse the previous implementation)
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        assert len(self.encodings['input_ids']) == len(self.labels), \
            "Mismatch between tokenized data and labels!"

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        try:
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item
        except IndexError as e:
            print(f"IndexError in __getitem__: {e}, idx: {idx}")
            raise

# Create datasets
train_dataset = CustomDataset(train_encodings, train_labels.reset_index(drop=True))
val_dataset = CustomDataset(val_encodings, val_labels.reset_index(drop=True))
test_dataset = CustomDataset(test_encodings, test_labels.reset_index(drop=True))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

print("Datasets and DataLoaders are ready.")

Datasets and DataLoaders are ready.


In [6]:
# Load the RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(set(train_labels)))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"RoBERTa model loaded and moved to {device}.")

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)

epochs = 7
num_training_steps = len(train_loader) * epochs
scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

print("Optimizer and scheduler are set.")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RoBERTa model loaded and moved to cuda.
Optimizer and scheduler are set.


In [7]:
# Training and Validation Loop
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    # Training phase
    model.train()
    train_loss = 0

    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {key: val.to(device) for key, val in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)
    print(f"Average Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {key: val.to(device) for key, val in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            predictions = torch.argmax(logits, dim=1)
            correct += (predictions == batch['labels']).sum().item()
            total += batch['labels'].size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



Epoch 1/7


Epoch 1: 100%|██████████| 235/235 [01:05<00:00,  3.58it/s, loss=2.32]


Average Training Loss: 1.8560
Validation Loss: 1.3301, Validation Accuracy: 0.6624

Epoch 2/7


Epoch 2: 100%|██████████| 235/235 [01:07<00:00,  3.50it/s, loss=0.744]


Average Training Loss: 0.9367
Validation Loss: 1.1407, Validation Accuracy: 0.7232

Epoch 3/7


...

Average Training Loss: 0.2287
Validation Loss: 1.1421, Validation Accuracy: 0.7472

Epoch 6/7


Epoch 6: 100%|██████████| 235/235 [01:06<00:00,  3.53it/s, loss=0.344]


Average Training Loss: 0.1261
Validation Loss: 1.1521, Validation Accuracy: 0.7376

Epoch 7/7


Epoch 7: 100%|██████████| 235/235 [01:06<00:00,  3.53it/s, loss=0.34]


Average Training Loss: 0.0891
Validation Loss: 1.1367, Validation Accuracy: 0.7488


In [8]:
# Test the model
model.eval()
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        batch = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        test_loss += loss.item()
        predictions = torch.argmax(logits, dim=1)
        correct += (predictions == batch['labels']).sum().item()
        total += batch['labels'].size(0)

avg_test_loss = test_loss / len(test_loader)
test_accuracy = correct / total

print(f"\nTest Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Test Loss: 1.1044
Test Accuracy: 0.7632


In [9]:
def predict_top5_categories_roberta(text, model, tokenizer, device, reverse_label_map):
    """
    Predict the top 5 categories for a given text using a fine-tuned RoBERTa model.

    Args:
        text (str): The input text to classify.
        model (torch.nn.Module): The fine-tuned RoBERTa model.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer used with the model.
        device (torch.device): The device (CPU/GPU) where the model is loaded.
        reverse_label_map (dict): Reverse mapping of numerical labels to category names.

    Returns:
        list: Top 5 categories with their probabilities.
    """
    # Tokenize the input text
    encoding = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"  # Return PyTorch tensors
    )

    # Move tensors to the device
    encoding = {key: val.to(device) for key, val in encoding.items()}

    # Ensure the model is in evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits

    # Calculate probabilities using softmax
    probabilities = torch.softmax(logits, dim=1).squeeze().tolist()

    # Get top 5 probabilities and their indices
    top5_indices = torch.topk(torch.tensor(probabilities), 5).indices.tolist()
    top5_probs = torch.topk(torch.tensor(probabilities), 5).values.tolist()
    # Map indices to original category names
    top5_categories = [(reverse_label_map[idx], prob) for idx, prob in zip(top5_indices, top5_probs)]

    return top5_categories

# Example input text
input_text = "Mbappe scored on his return from a thigh problem as Real Madrid beat Pachuca in midweek to win the Intercontinental Cup."
# Predict top 5 categories using RoBERTa
top5_results = predict_top5_categories_roberta(input_text, model, tokenizer, device, reverse_label_map)

# Display the results
if top5_results:
    print("\nTop 5 Probable Categories:")
    for category, prob in top5_results:
        print(f"{category}: {prob:.4f}")



Top 5 Probable Categories:
Sport: 0.0980
Games: 0.0458
Commerce: 0.0456
News: 0.0446
Style: 0.0433


In [10]:
import os
import torch

# Define the save directory
save_directory = "/content/drive/MyDrive/Saved Model"
os.makedirs(save_directory, exist_ok=True)  # Ensure the directory exists

# Save RoBERTa model's state dictionary
model_save_path = f"{save_directory}/roberta_finetuned_model.pt"
torch.save(model.state_dict(), model_save_path)

print(f"RoBERTa model state dictionary saved to: {model_save_path}")


RoBERTa model state dictionary saved to: /content/drive/MyDrive/Saved Model/roberta_finetuned_model.pt


**Finetuning Using DeBERTa**

Step-1: Load the DeBERTa Model and Tokenizer

In [11]:
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, AdamW, get_scheduler

# Load the pre-trained DeBERTa tokenizer and model
tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-base')
model = DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', num_labels=len(set(train_labels)))

# Move the model to GPU/CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"DeBERTa model and tokenizer loaded, moved to {device}.")



tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DeBERTa model and tokenizer loaded, moved to cuda.


Step-2: Tokenize the Data

In [12]:
def tokenize_data(texts):
    return tokenizer(
        list(texts),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )

train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)
test_encodings = tokenize_data(test_texts)

print("Tokenization completed.")


Tokenization completed.


Step 3: Define the Dataset Class

In [13]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        assert len(self.encodings['input_ids']) == len(self.labels), \
            "Mismatch between tokenized data and labels!"

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item


Step 4: Prepare DataLoaders

In [14]:
# Reset indices for labels
train_labels = train_labels.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

# Create datasets and DataLoaders
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

print("Datasets and DataLoaders are ready.")


Datasets and DataLoaders are ready.


Step 5: Define the Optimizer and Scheduler

In [15]:
optimizer = AdamW(model.parameters(), lr=2e-5)  # Use a smaller learning rate for DeBERTa
epochs = 7
num_training_steps = len(train_loader) * epochs

scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

print("Optimizer and scheduler set.")


Optimizer and scheduler set.


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step 6: Training Loop

In [16]:
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    # Training phase
    model.train()
    train_loss = 0

    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {key: val.to(device) for key, val in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)
    print(f"Average Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            predictions = torch.argmax(logits, dim=1)
            correct += (predictions == batch['labels']).sum().item()
            total += batch['labels'].size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



Epoch 1/7


Epoch 1: 100%|██████████| 235/235 [01:26<00:00,  2.72it/s, loss=1.89]


Average Training Loss: 2.8200
Validation Loss: 2.1095, Validation Accuracy: 0.4784

Epoch 2/7


Epoch 2: 100%|██████████| 235/235 [01:24<00:00,  2.77it/s, loss=1.35]


Average Training Loss: 1.7195
Validation Loss: 1.6122, Validation Accuracy: 0.6016

Epoch 3/7


...

Average Training Loss: 0.6435
Validation Loss: 1.2020, Validation Accuracy: 0.7232

Epoch 6/7


Epoch 6: 100%|██████████| 235/235 [01:24<00:00,  2.77it/s, loss=0.44]


Average Training Loss: 0.5152
Validation Loss: 1.2187, Validation Accuracy: 0.7168

Epoch 7/7


Epoch 7: 100%|██████████| 235/235 [01:24<00:00,  2.77it/s, loss=0.729]


Average Training Loss: 0.4477
Validation Loss: 1.1942, Validation Accuracy: 0.7248


Step 7: Testing the Model

In [17]:
model.eval()
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        batch = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        test_loss += loss.item()
        predictions = torch.argmax(logits, dim=1)
        correct += (predictions == batch['labels']).sum().item()
        total += batch['labels'].size(0)

avg_test_loss = test_loss / len(test_loader)
test_accuracy = correct / total

print(f"\nTest Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



Test Loss: 1.1861
Test Accuracy: 0.7648


Step 8: Save the model

In [18]:
import os
import torch

# Define the save directory
save_directory = "/content/drive/MyDrive/Saved Model"
os.makedirs(save_directory, exist_ok=True)  # Ensure the directory exists

# Save RoBERTa model's state dictionary
model_save_path = f"{save_directory}/deberta_finetuned_model.pt"
torch.save(model.state_dict(), model_save_path)

print(f"DeBERTa model state dictionary saved to: {model_save_path}")


DeBERTa model state dictionary saved to: /content/drive/MyDrive/Saved Model/deberta_finetuned_model.pt


**Finetuning With ALBERT**

Step 1: Load the ALBERT Model and Tokenizer

In [19]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification, AdamW, get_scheduler

# Load the pre-trained ALBERT tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=len(set(train_labels)))

# Move the model to GPU/CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"ALBERT model and tokenizer loaded, moved to {device}.")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ALBERT model and tokenizer loaded, moved to cuda.


Step 2: Tokenize the Data

In [20]:
def tokenize_data(texts):
    return tokenizer(
        list(texts),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )

train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)
test_encodings = tokenize_data(test_texts)

print("Tokenization completed.")


Tokenization completed.


Step 3: Define the Dataset Class

In [21]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        assert len(self.encodings['input_ids']) == len(self.labels), \
            "Mismatch between tokenized data and labels!"

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item





Step 4: Prepare DataLoaders

In [22]:
# Reset indices for labels
train_labels = train_labels.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

# Create datasets and DataLoaders
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

print("Datasets and DataLoaders are ready.")


Datasets and DataLoaders are ready.


Step 5: Define the Optimizer and Scheduler

In [23]:
optimizer = AdamW(model.parameters(), lr=1e-5)  # ALBERT often benefits from smaller learning rates
epochs = 7
num_training_steps = len(train_loader) * epochs

scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

print("Optimizer and scheduler set.")


Optimizer and scheduler set.


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step 6: Training Loop

In [24]:
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    # Training phase
    model.train()
    train_loss = 0

    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {key: val.to(device) for key, val in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)
    print(f"Average Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            predictions = torch.argmax(logits, dim=1)
            correct += (predictions == batch['labels']).sum().item()
            total += batch['labels'].size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



Epoch 1/7


Epoch 1: 100%|██████████| 235/235 [01:06<00:00,  3.52it/s, loss=2.72]


Average Training Loss: 2.8517
Validation Loss: 2.3212, Validation Accuracy: 0.4240

Epoch 2/7


Epoch 2: 100%|██████████| 235/235 [01:06<00:00,  3.56it/s, loss=2.22]


Average Training Loss: 1.9539
Validation Loss: 1.7350, Validation Accuracy: 0.5760

Epoch 3/7


...

Average Training Loss: 1.0453
Validation Loss: 1.3066, Validation Accuracy: 0.6672

Epoch 6/7


Epoch 6: 100%|██████████| 235/235 [01:06<00:00,  3.56it/s, loss=0.295]


Average Training Loss: 0.9240
Validation Loss: 1.2818, Validation Accuracy: 0.6752

Epoch 7/7


Epoch 7: 100%|██████████| 235/235 [01:06<00:00,  3.55it/s, loss=0.421]


Average Training Loss: 0.8405
Validation Loss: 1.2745, Validation Accuracy: 0.6784


Step 7: Testing the Model

In [25]:
model.eval()
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        batch = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        test_loss += loss.item()
        predictions = torch.argmax(logits, dim=1)
        correct += (predictions == batch['labels']).sum().item()
        total += batch['labels'].size(0)

avg_test_loss = test_loss / len(test_loader)
test_accuracy = correct / total

print(f"\nTest Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



Test Loss: 1.3041
Test Accuracy: 0.7040


Step 8: Save ALBERT model

In [26]:
import os
import torch

# Define the save directory
save_directory = "/content/drive/MyDrive/Saved Model"
os.makedirs(save_directory, exist_ok=True)  # Ensure the directory exists

# Save RoBERTa model's state dictionary
model_save_path = f"{save_directory}/albert_finetuned_model.pt"
torch.save(model.state_dict(), model_save_path)

print(f"ALBERT model state dictionary saved to: {model_save_path}")


ALBERT model state dictionary saved to: /content/drive/MyDrive/Saved Model/albert_finetuned_model.pt


**Finetuning using T5 (Text-To-Text Transfer Transformer)**

Step 1: Load the T5 Model and Tokenizer

In [27]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_scheduler


# Load the pre-trained T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Move the model to GPU/CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"T5 model and tokenizer loaded, moved to {device}.")


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5 model and tokenizer loaded, moved to cuda.


Step 2: Prepare the Data for T5

In [32]:
# Dynamically create a label_map from dataset labels
dataset_labels = set(train_labels) | set(val_labels) | set(test_labels)
label_map = {label: f"Category_{label}" for label in dataset_labels}

# Reverse the label_map
reverse_label_map = {v: k for k, v in label_map.items()}

print("Updated label_map:", label_map)
print("Updated reverse_label_map:", reverse_label_map)

Updated label_map: {0: 'Category_0', 1: 'Category_1', 2: 'Category_2', 3: 'Category_3', 4: 'Category_4', 5: 'Category_5', 6: 'Category_6', 7: 'Category_7', 8: 'Category_8', 9: 'Category_9', 10: 'Category_10', 11: 'Category_11', 12: 'Category_12', 13: 'Category_13', 14: 'Category_14', 15: 'Category_15', 16: 'Category_16', 17: 'Category_17', 18: 'Category_18', 19: 'Category_19', 20: 'Category_20', 21: 'Category_21', 22: 'Category_22', 23: 'Category_23', 24: 'Category_24', 25: 'Category_25', 26: 'Category_26', 27: 'Category_27', 28: 'Category_28', 29: 'Category_29', 30: 'Category_30', 31: 'Category_31'}
Updated reverse_label_map: {'Category_0': 0, 'Category_1': 1, 'Category_2': 2, 'Category_3': 3, 'Category_4': 4, 'Category_5': 5, 'Category_6': 6, 'Category_7': 7, 'Category_8': 8, 'Category_9': 9, 'Category_10': 10, 'Category_11': 11, 'Category_12': 12, 'Category_13': 13, 'Category_14': 14, 'Category_15': 15, 'Category_16': 16, 'Category_17': 17, 'Category_18': 18, 'Category_19': 19, 'Cat

In [33]:
# Prepare data in T5 format
def prepare_data(texts, labels, label_map):
    input_texts = [f"Classify: {text}" for text in texts]
    try:
        target_texts = [label_map[label] for label in labels]
    except KeyError as e:
        print(f"KeyError: Label {e} not found in label_map.")
        print("Check your dataset and label_map for inconsistencies.")
        raise
    return input_texts, target_texts

# Prepare input and target texts
label_map = {v: k for k, v in reverse_label_map.items()}  # Reverse the label map
train_inputs, train_targets = prepare_data(train_texts, train_labels, label_map)
val_inputs, val_targets = prepare_data(val_texts, val_labels, label_map)
test_inputs, test_targets = prepare_data(test_texts, test_labels, label_map)


Step 3: Tokenize the Data

In [34]:
# Tokenize the data for T5
def tokenize_data(inputs, targets, tokenizer, max_length=128):
    encodings = tokenizer(
        inputs,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    target_encodings = tokenizer(
        targets,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return encodings, target_encodings

train_encodings, train_target_encodings = tokenize_data(train_inputs, train_targets, tokenizer)
val_encodings, val_target_encodings = tokenize_data(val_inputs, val_targets, tokenizer)
test_encodings, test_target_encodings = tokenize_data(test_inputs, test_targets, tokenizer)

print("Tokenization completed.")


Tokenization completed.


Step 4: Define the Dataset Class

In [35]:
class CustomT5Dataset(Dataset):
    def __init__(self, encodings, target_encodings):
        self.encodings = encodings
        self.target_encodings = target_encodings
        assert len(self.encodings['input_ids']) == len(self.target_encodings['input_ids']), \
            "Mismatch between input and target data!"

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.target_encodings['input_ids'][idx]
        return item


Step 5: Prepare DataLoaders

In [36]:
# Create datasets and DataLoaders
train_dataset = CustomT5Dataset(train_encodings, train_target_encodings)
val_dataset = CustomT5Dataset(val_encodings, val_target_encodings)
test_dataset = CustomT5Dataset(test_encodings, test_target_encodings)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

print("Datasets and DataLoaders are ready.")


Datasets and DataLoaders are ready.


Step 6: Define the Optimizer and Scheduler

In [37]:
optimizer = AdamW(model.parameters(), lr=3e-5)  # T5 works well with slightly larger learning rates
epochs = 7
num_training_steps = len(train_loader) * epochs

scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

print("Optimizer and scheduler set.")


Optimizer and scheduler set.


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step 7: Training Loop

In [38]:
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    # Training phase
    model.train()
    train_loss = 0

    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {key: val.to(device) for key, val in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)
    print(f"Average Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss:.4f}")



Epoch 1/7


Epoch 1: 100%|██████████| 235/235 [01:46<00:00,  2.22it/s, loss=0.789]


Average Training Loss: 1.4161
Validation Loss: 0.8218

Epoch 2/7


Epoch 2: 100%|██████████| 235/235 [01:45<00:00,  2.22it/s, loss=0.772]


Average Training Loss: 0.8470
Validation Loss: 0.7699

Epoch 3/7


...

Average Training Loss: 0.5938
Validation Loss: 0.4640

Epoch 6/7


Epoch 6: 100%|██████████| 235/235 [01:45<00:00,  2.22it/s, loss=0.443]


Average Training Loss: 0.5365
Validation Loss: 0.4231

Epoch 7/7


Epoch 7: 100%|██████████| 235/235 [01:45<00:00,  2.22it/s, loss=0.543]


Average Training Loss: 0.5116
Validation Loss: 0.4139


Step 8: Testing the Model

In [39]:
model.eval()
test_loss = 0

with torch.no_grad():
    for batch in test_loader:
        batch = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**batch)
        test_loss += outputs.loss.item()

avg_test_loss = test_loss / len(test_loader)

print(f"\nTest Loss: {avg_test_loss:.4f}")



Test Loss: 0.4149


Updated Code for T5 Evaluation (Loss and Accuracy)

In [42]:
#need to work with it's accuracy why it is less
model.eval()
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        # Move data to the device
        batch = {key: val.to(device) for key, val in batch.items()}

        # Forward pass to calculate loss
        outputs = model(**batch)
        test_loss += outputs.loss.item()

        # Generate predictions using beam search
        predicted_ids = model.generate(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            max_length=16,
            num_beams=5  # Use beam search
        )

        # Decode predictions and true labels to text
        predicted_texts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
        true_texts = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

        # Compare predictions with true labels (relaxed matching)
        correct += sum(pred.strip().lower() == true.strip().lower() for pred, true in zip(predicted_texts, true_texts))
        total += len(true_texts)

# Calculate average test loss
avg_test_loss = test_loss / len(test_loader)

# Calculate accuracy
test_accuracy = correct / total

# Print results
print(f"\nTest Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



Test Loss: 0.4149
Test Accuracy: 0.5696


Step 9: Save T5 Model

In [43]:
import os
import torch

# Define the save directory
save_directory = "/content/drive/MyDrive/Saved Model"
os.makedirs(save_directory, exist_ok=True)  # Ensure the directory exists

# Save RoBERTa model's state dictionary
model_save_path = f"{save_directory}/t5_finetuned_model.pt"
torch.save(model.state_dict(), model_save_path)

print(f"T5 model state dictionary saved to: {model_save_path}")


T5 model state dictionary saved to: /content/drive/MyDrive/Saved Model/t5_finetuned_model.pt
